<a href="https://colab.research.google.com/github/AarZheng/ai-agent-cs-honor/blob/main/Module_1_AI_Agent_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install langchain wikipedia google-generativeai langchain-community langchain-google-genai beautifulsoup4

  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 7.8 MB/s eta 0:00:00
   ━

# Connect the Gemini API


In [2]:
import google.generativeai as genai
from google.colab import userdata

# Load the API key from Colab secrets
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

# Configure the Gemini API
genai.configure(api_key=GOOGLE_API_KEY)

# Build a basic langchain agent

In [3]:
from langchain_community.document_loaders import WebBaseLoader

def LoadWebPage(url: str) -> str:
  """Loads content from a web page given a URL."""
  loader = WebBaseLoader(url)
  docs = loader.load()
  # Assuming we only care about the content of the first document
  if docs:
    return docs[0].page_content
  else:
    return "Could not load content from the URL."

In [4]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain_community.document_loaders import WebBaseLoader

#Instantiate the Wikipedia tool
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=1000)
wikipedia_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

#Initialize the Gemini model, explicitly passing the API key and using a supported model
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0, google_api_key=GOOGLE_API_KEY)

# Define the LoadWebPage function (if not already defined)
def LoadWebPage(url: str) -> str:
  """Loads content from a web page given a URL."""
  loader = WebBaseLoader(url)
  docs = loader.load()
  # Assuming we only care about the content of the first document
  if docs:
    return docs[0].page_content
  else:
    return "Could not load content from the URL."

# Create a tool that uses the LoadWebPage function with a hardcoded URL from Buzzfeed
food_tool = Tool(
    name="Food Tool",
    func=lambda x: LoadWebPage("https://www.buzzfeed.com/hannahloewentheil/people-sharing-most-iconic-foods-in-america"),
    description="Reads the content of a specific hardcoded URL."
)

#Initialize the LangChain agent and handle parsing errors
agent = initialize_agent(
    tools=[wikipedia_tool, food_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True # Add this line to handle parsing errors
)

/tmp/ipython-input-2218207238.py:34: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


# Run the agent

In [5]:
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain.tools import Tool
from google.colab import userdata

# Load the GOOGLE_CSE_ID from Colab secrets
GOOGLE_CSE_ID = userdata.get('GOOGLE_CSE_ID')

# Instantiate the Google Search Tool using GoogleSearchAPIWrapper
search = GoogleSearchAPIWrapper(google_cse_id=GOOGLE_CSE_ID, google_api_key=GOOGLE_API_KEY)
search_tool = Tool(
    name="Google Search",
    description="Search Google for recent results.",
    func=search.run,
)

# Add the search tool to the existing tools list
tools = [wikipedia_tool, food_tool, search_tool]

# Re-initialize the agent with the updated tools list
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

SecretNotFoundError: Secret GOOGLE_CSE_ID does not exist.

In [ ]:
import os

def write_to_file(filename: str, content: str):
  """Writes content to a file."""
  try:
    with open(filename, 'w') as f:
      f.write(content)
    return f"Successfully wrote to {filename}"
  except Exception as e:
    return f"Error writing to file: {e}"

# Create a tool that uses the write_to_file function
file_writer_tool = Tool(
    name="File Writer",
    func=write_to_file,
    description="Writes text content to a file. Input should be a comma-separated string of filename and content."
)

# Add the file writer tool to the existing tools list
# Ensure 'tools' list is defined from previous steps, assuming it includes wikipedia_tool, food_tool, and search_tool
tools = [wikipedia_tool, food_tool, search_tool, file_writer_tool]

# Re-initialize the agent with the updated tools list
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

In [ ]:
pip install serpapi

In [ ]:
from serpapi import GoogleSearch

def google_search(query, num_results=5):
    params = {
        "engine": "google",
        "q": query,
        "num": num_results,
        "api_key": "YOUR_SERPAPI_KEY"
    }
    search = GoogleSearch(params)
    results = search.get_dict()

    summaries = []
    for result in results.get("organic_results", []):
        title = result.get("title")
        link = result.get("link")
        snippet = result.get("snippet", "")
        summaries.append(f"{title} — {snippet}\n{link}")
    return "\n\n".join(summaries)

In [ ]:
tools = [wikipedia_tool, food_tool, google_search_tool, file_writer_tool]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

In [ ]:
food_response = agent.run("Using the food tool, find iconic foods for California and choose one of them.")
print(f"The agent chose: {food_response}")

# description = agent.run("Using the wikipedia tool, find information on " + food_response)

agent.run("Using the google search tool, find link to the first recipe for " + food_response)